# Find the most limiting nutrient at each phytoplankton group
* need to create all of the limitating nutrients in a single file for each phytoplankton group. 
* find the most limiting and create into a limiting file. 
Then: 
* calculate the temperature limitation term
* need to create all of the limitation factors in a single file for each phytoplankton gtoup. 
* find the value that is the minimum from all of the limitation terms
* make each limitation term a differnt color
* plot the color that is most limiting at every month for every phyto!

In [13]:
%matplotlib inline
import warnings
warnings.simplefilter("ignore") # Silence warnings
from mpl_toolkits.basemap import Basemap
import hvplot.xarray
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import numpy as np
from matplotlib.pyplot import figure
import pylab 

### 1990 to 2009 monthly data
this data set contains: 
* global data at every month from 1990 to 2009
* global data at every depth level from 0 meters to 4000meters. 
* every variable in the model. 

In [2]:
ds = xr.open_dataset('/glade/work/gabyn/MARBL_spectra/remapgrid.out1.nc')

## Calcualte the temperature limiation term! 
$$r = r_o e^{\frac{-E_a(T_o -T)}{k T_o T}}$$
$$\frac{\mu}{\mu_o} = e^{\frac{-E_a(T_o -T)}{k T_o T}}$$ 

to simplify however, we will create $\alpha = \frac{E_a}{k}$ and $\gamma_T = 
\frac{\mu}{\mu_o}$ so that: 
$$\gamma_T = e^{ \frac{-\alpha(T_o - T)}{T_o T}}$$

$T_o = 298.15 ^oK$
* it is close to the temperature at which many physiological experiments are run, and it is the mean temperature of the subtropical gyres, which represent a large proportion of the world’s open oceans.

$k = 8.617e^{-5} eVK^{-1}$ 

$E_a = 0.317$ phytoplankton (Kremer et al., 2017).

$E_a = 0.42$ picoplankton 
* Multiple studies have shown that picoplankton have a higher temperature sensitivity compared to phytoplankton of larger sizes (Chen et al., 2014; Stawiarski et al., 2016).

$E_a = 0.65$ zooplankton 
* MTE predicts that heterotrophic organisms will have a greater temperature dependence than autotrophs, as Rubisco carboxylation (rate limiting for photosynthesis) has a lower activation energy (Ea) than ATP synthesis (Allen et al., 2005; Lopez-Urrutia et al., 2006).

and we will have to convert all the surface temperatures from $^oC$ to $K$ and the temperature on the equation and then solve for every gorup... 
$$0°C + 273.15 = 273.15K$$


In [3]:
# convert celcius to kelvin 
temp_K = ds.TEMP + 273.15

# constants 
𝑘=8.617e-5
𝐸𝑎_phyto = 0.317 
𝐸𝑎_pico  = 0.42
𝐸𝑎_zoo   = 0.65
𝑇𝑜 = 298.15

# create simpler variables. 
𝛼_phyto = 𝐸𝑎_phyto/𝑘
𝛼_pico = 𝐸𝑎_pico/𝑘
𝛼_zoo = 𝐸𝑎_zoo/𝑘

# calculate temperature limitation term
𝛾𝑇_phyto = np.exp((-𝛼_phyto*(𝑇𝑜-temp_K))/(𝑇𝑜*temp_K))
𝛾𝑇_pico = np.exp((-𝛼_pico*(𝑇𝑜-temp_K))/(𝑇𝑜*temp_K))
𝛾𝑇_zoo = np.exp((-𝛼_zoo*(𝑇𝑜-temp_K))/(𝑇𝑜*temp_K))

# limitating nutrients

In [4]:
diat1_surf_nutrient = np.stack([ds.diat1_Fe_lim_surf, ds.diat1_N_lim_surf, 
                       ds.diat1_P_lim_surf, ds.diat1_SiO3_lim_surf])

diat2_surf_nutrient = np.stack([ds.diat2_Fe_lim_surf, ds.diat2_N_lim_surf, 
                       ds.diat2_P_lim_surf, ds.diat2_SiO3_lim_surf])

diat3_surf_nutrient = np.stack([ds.diat3_Fe_lim_surf, ds.diat3_N_lim_surf, 
                       ds.diat3_P_lim_surf, ds.diat3_SiO3_lim_surf])

mp1_surf_nutrient = np.stack([ds.mp1_Fe_lim_surf, ds.mp1_N_lim_surf, 
                     ds.mp1_P_lim_surf])

mp2_surf_nutrient = np.stack([ds.mp2_Fe_lim_surf, ds.mp2_N_lim_surf, 
                     ds.mp2_P_lim_surf])

mp3_surf_nutrient = np.stack([ds.mp3_Fe_lim_surf, ds.mp3_N_lim_surf, 
                     ds.mp3_P_lim_surf])

mp4_surf_nutrient = np.stack([ds.mp4_Fe_lim_surf, ds.mp4_N_lim_surf, 
                     ds.mp4_P_lim_surf])

diaz_surf_nutrient = np.stack([ds.diaz_Fe_lim_surf, ds.diaz_P_lim_surf])

pp_surf_nutrient = np.stack([ds.pp_Fe_lim_surf, ds.pp_N_lim_surf, 
                    ds.pp_P_lim_surf])

## Find the most limiting factor at each point. 
I want to make a loop that finds the minimum of the limitation terms at every lat and lon and month, and gives an indece of which one it is, then makes that a number from 1-5 where each number will be a different color. 

In [5]:
diat1_surf_nutrient[diat1_surf_nutrient>0.9]= np.nan
diat2_surf_nutrient[diat2_surf_nutrient>0.9]= np.nan
diat3_surf_nutrient[diat3_surf_nutrient>0.9]= np.nan
mp1_surf_nutrient[mp1_surf_nutrient>0.9]= np.nan
mp2_surf_nutrient[mp2_surf_nutrient>0.9]= np.nan
mp3_surf_nutrient[mp3_surf_nutrient>0.9]= np.nan
mp4_surf_nutrient[mp4_surf_nutrient>0.9]= np.nan
pp_surf_nutrient[pp_surf_nutrient>0.9]= np.nan
diaz_surf_nutrient[diaz_surf_nutrient>0.9]= np.nan

In [16]:
def _nanargmin(arr, axis):
...    try:
...       return np.nanargmin(arr, axis)
...    except ValueError:
...       return np.nan

In [29]:
# diatom 1 
diat1_most_lim_surf_nutrient = np.zeros((12,180,360)) 
diat1_index_nutrient = np.zeros((12,180,360)) 
for j in range(12):
    for k in range(180):
        for l in range(360):
            diat1_most_lim_surf_nutrient[j,k,l] = np.nanmin(diat1_surf_nutrient[:,j, k, l])
            diat1_index_nutrient[j,k,l] = _nanargmin(diat1_surf_nutrient[:,j, k, l], axis=0)


# diatom 2
diat2_most_lim_surf_nutrient = np.zeros((12,180,360)) 
diat2_index_nutrient = np.zeros((12,180,360)) 
for j in range(12):
    for k in range(180):
        for l in range(360):
            diat2_index_nutrient[j,k,l] = _nanargmin(diat2_surf_nutrient[:,j, k, l], axis=0)
            diat2_most_lim_surf_nutrient[j,k,l] = np.nanmin(diat2_surf_nutrient[:,j, k, l])


# diatom 3
diat3_most_lim_surf_nutrient = np.zeros((12,180,360)) 
diat3_index_nutrient = np.zeros((12,180,360)) 
for j in range(12):
    for k in range(180):
        for l in range(360):
            diat3_index_nutrient[j,k,l] = _nanargmin(diat3_surf_nutrient[:,j, k, l], axis=0)
            diat3_most_lim_surf_nutrient[j,k,l] = np.nanmin(diat3_surf_nutrient[:,j, k, l])
            if np.isnan(diat3_most_lim_surf_nutrient[j,k,l]) == True:
                 diat3_index_nutrient[j,k,l] = np.nan


# mp1
mp1_most_lim_surf_nutrient = np.zeros((12,180,360)) 
mp1_index_nutrient = np.zeros((12,180,360)) 
for j in range(12):
    for k in range(180):
        for l in range(360):
            mp1_index_nutrient[j,k,l] = _nanargmin(mp1_surf_nutrient[:,j, k, l], axis=0)
            mp1_most_lim_surf_nutrient[j,k,l] = np.nanmin(mp1_surf_nutrient[:,j, k, l])
            if np.isnan(mp1_most_lim_surf_nutrient[j,k,l]) == True:
                mp1_index_nutrient[j,k,l] = np.nan
            

# mp2
mp2_most_lim_surf_nutrient = np.zeros((12,180,360)) 
mp2_index_nutrient = np.zeros((12,180,360)) 
for j in range(12):
    for k in range(180):
        for l in range(360):
            mp2_index_nutrient[j,k,l] = _nanargmin(mp2_surf_nutrient[:,j, k, l], axis=0)  
            mp2_most_lim_surf_nutrient[j,k,l] = np.nanmin(mp2_surf_nutrient[:,j, k, l])
            if np.isnan(mp2_most_lim_surf_nutrient[j,k,l]) == True:
                mp2_index_nutrient[j,k,l] = np.nan
            
            
# mp3
mp3_most_lim_surf_nutrient = np.zeros((12,180,360)) 
mp3_index_nutrient = np.zeros((12,180,360)) 
for j in range(12):
    for k in range(180):
        for l in range(360):
            mp3_index_nutrient[j,k,l] = _nanargmin(mp3_surf_nutrient[:,j, k, l], axis=0)
            mp3_most_lim_surf_nutrient[j,k,l] = np.nanmin(mp3_surf_nutrient[:,j, k, l])
            if np.isnan(mp3_most_lim_surf_nutrient[j,k,l]) == True:
                mp3_index_nutrient[j,k,l] = np.nan
            
# mp4
mp4_most_lim_surf_nutrient = np.zeros((12,180,360)) 
mp4_index_nutrient = np.zeros((12,180,360)) 
for j in range(12):
    for k in range(180):
        for l in range(360):
            mp4_index_nutrient[j,k,l] = _nanargmin(mp4_surf_nutrient[:,j, k, l], axis=0)
            mp4_most_lim_surf_nutrient[j,k,l] = np.nanmin(mp4_surf_nutrient[:,j, k, l])
            if np.isnan(mp4_most_lim_surf_nutrient[j,k,l]) == True:
                mp4_index_nutrient[j,k,l] = np.nan

# pp
pp_most_lim_surf_nutrient = np.zeros((12,180,360)) 
pp_index_nutrient = np.zeros((12,180,360)) 
for j in range(12):
    for k in range(180):
        for l in range(360):
            pp_index_nutrient[j,k,l] = _nanargmin(pp_surf_nutrient[:,j, k, l], axis=0)
            pp_most_lim_surf_nutrient[j,k,l] = np.nanmin(pp_surf_nutrient[:,j, k, l])
            if np.isnan(pp_most_lim_surf_nutrient[j,k,l]) == True:
                pp_index_nutrient[j,k,l] = np.nan
            
# diaz
diaz_most_lim_surf_nutrient = np.zeros((12,180,360)) 
diaz_index_nutrient = np.zeros((12,180,360)) 
for j in range(12):
    for k in range(180):
        for l in range(360):
            diaz_index_nutrient[j,k,l] = _nanargmin(diaz_surf_nutrient[:,j, k, l], axis=0)
            diaz_most_lim_surf_nutrient[j,k,l] = np.nanmin(diaz_surf_nutrient[:,j, k, l])
            if np.isnan(diaz_most_lim_surf_nutrient[j,k,l]) == True:
                diaz_index_nutrient[j,k,l] = np.nan

In [31]:
import scipy.io as sio
sio.savemat('most_limiting_surf_nutrient.mat',{'diaz_most_lim_surf_nutrient':diaz_most_lim_surf_nutrient, 'pp_most_lim_surf_nutrient':pp_most_lim_surf_nutrient,
                                      'diat1_most_lim_surf_nutrient':diat1_most_lim_surf_nutrient,'diat2_most_lim_surf_nutrient':diat2_most_lim_surf_nutrient,
                                     'diat3_most_lim_surf_nutrient':diat3_most_lim_surf_nutrient,'mp1_most_lim_surf_nutrient':mp1_most_lim_surf_nutrient,
                                     'mp2_most_lim_surf_nutrient':mp2_most_lim_surf_nutrient,'mp3_most_lim_surf_nutrient':mp3_most_lim_surf_nutrient,
                                     'mp4_most_lim_surf_nutrient':mp4_most_lim_surf_nutrient})
sio.savemat('most_limiting_surf_index_nutrient.mat', {'diaz_index_nutrient':diaz_index_nutrient, 'pp_index_nutrient':pp_index_nutrient, 'mp4_index_nutrient':mp4_index_nutrient,
                                            'mp3_index_nutrient':mp3_index_nutrient, 'mp2_index_nutrient':mp2_index_nutrient, 'mp1_index_nutrient':mp1_index_nutrient,
                                            'diat1_index_nutrient':diat1_index_nutrient, 'diat2_index_nutrient':diat2_index_nutrient, 'diat3_index_nutrient':diat3_index_nutrient})

# limiting factors

In [32]:
diat1_surf = np.stack([ds.diat1_light_lim_surf, 
                       𝛾𝑇_phyto[:,0,:,:], diat1_most_lim_surf_nutrient])

diat2_surf = np.stack([ds.diat2_light_lim_surf, 
                       𝛾𝑇_phyto[:,0,:,:], diat2_most_lim_surf_nutrient])

diat3_surf = np.stack([ds.diat3_light_lim_surf, 
                       𝛾𝑇_phyto[:,0,:,:], diat3_most_lim_surf_nutrient])

mp1_surf = np.stack([ds.mp1_light_lim_surf, 
                     𝛾𝑇_phyto[:,0,:,:], mp1_most_lim_surf_nutrient])

mp2_surf = np.stack([ds.mp2_light_lim_surf, 
                     𝛾𝑇_phyto[:,0,:,:], mp2_most_lim_surf_nutrient])

mp3_surf = np.stack([ds.mp3_light_lim_surf, 
                     𝛾𝑇_phyto[:,0,:,:], mp3_most_lim_surf_nutrient])

mp4_surf = np.stack([ds.mp4_light_lim_surf, 
                     𝛾𝑇_phyto[:,0,:,:], mp4_most_lim_surf_nutrient])

diaz_surf = np.stack([ ds.diaz_light_lim_surf, 𝛾𝑇_phyto[:,0,:,:], 
                      diaz_most_lim_surf_nutrient])

pp_surf = np.stack([ds.pp_light_lim_surf, 
                    𝛾𝑇_pico[:,0,:,:], pp_most_lim_surf_nutrient])

## Find the most limiting factor at each point. 
I want to make a loop that finds the minimum of the limitation terms at every lat and lon and month, and gives an indece of which one it is, then makes that a number from 1-5 where each number will be a different color. 

In [33]:
diat1_surf[diat1_surf>0.9]= np.nan
diat2_surf[diat2_surf>0.9]= np.nan
diat3_surf[diat3_surf>0.9]= np.nan
mp1_surf[mp1_surf>0.9]= np.nan
mp2_surf[mp2_surf>0.9]= np.nan
mp3_surf[mp3_surf>0.9]= np.nan
mp4_surf[mp4_surf>0.9]= np.nan
pp_surf[pp_surf>0.9]= np.nan
diaz_surf[diaz_surf>0.9]= np.nan

In [34]:
# diatom 1 
diat1_most_lim_surf = np.zeros((12,180,360)) 
diat1_index = np.zeros((12,180,360)) 
for j in range(12):
    for k in range(180):
        for l in range(360):
            diat1_most_lim_surf[j,k,l] = np.nanmin(diat1_surf[:,j, k, l])
            diat1_index[j,k,l] = _nanargmin(diat1_surf[:,j, k, l], axis=0)
            

# diatom 2
diat2_most_lim_surf = np.zeros((12,180,360)) 
diat2_index = np.zeros((12,180,360)) 
for j in range(12):
    for k in range(180):
        for l in range(360):   
            diat2_most_lim_surf[j,k,l] = np.nanmin(diat2_surf[:,j, k, l])
            diat2_index[j,k,l] = _nanargmin(diat2_surf[:,j, k, l], axis=0)


# diatom 3
diat3_most_lim_surf = np.zeros((12,180,360)) 
diat3_index = np.zeros((12,180,360)) 
for j in range(12):
    for k in range(180):
        for l in range(360):
            diat3_most_lim_surf[j,k,l] = np.nanmin(diat3_surf[:,j, k, l])
            diat3_index[j,k,l] = _nanargmin(diat3_surf[:,j, k, l], axis=0)
            


# mp1
mp1_most_lim_surf = np.zeros((12,180,360)) 
mp1_index = np.zeros((12,180,360)) 
for j in range(12):
    for k in range(180):
        for l in range(360):
            mp1_most_lim_surf[j,k,l] = np.nanmin(mp1_surf[:,j, k, l])
            mp1_index[j,k,l] = _nanargmin(mp1_surf[:,j, k, l], axis=0)
            

# mp2
mp2_most_lim_surf = np.zeros((12,180,360)) 
mp2_index = np.zeros((12,180,360)) 
for j in range(12):
    for k in range(180):
        for l in range(360): 
            mp2_most_lim_surf[j,k,l] = np.nanmin(mp2_surf[:,j, k, l])
            mp2_index[j,k,l] = _nanargmin(mp2_surf[:,j, k, l], axis=0)
            
            
# mp3
mp3_most_lim_surf = np.zeros((12,180,360)) 
mp3_index = np.zeros((12,180,360)) 
for j in range(12):
    for k in range(180):
        for l in range(360): 
            mp3_most_lim_surf[j,k,l] = np.nanmin(mp3_surf[:,j, k, l])
            mp3_index[j,k,l] = _nanargmin(mp3_surf[:,j, k, l], axis=0)
            
            
# mp4
mp4_most_lim_surf = np.zeros((12,180,360)) 
mp4_index = np.zeros((12,180,360)) 
for j in range(12):
    for k in range(180):
        for l in range(360):
            mp4_most_lim_surf[j,k,l] = np.nanmin(mp4_surf[:,j, k, l])
            mp4_index[j,k,l] = _nanargmin(mp4_surf[:,j, k, l], axis=0)
            

# pp
pp_most_lim_surf = np.zeros((12,180,360)) 
pp_index = np.zeros((12,180,360)) 
for j in range(12):
    for k in range(180):
        for l in range(360):
            pp_most_lim_surf[j,k,l] = np.nanmin(pp_surf[:,j, k, l])
            pp_index[j,k,l] = _nanargmin(pp_surf[:,j, k, l], axis=0)
            
            
# diaz
diaz_most_lim_surf = np.zeros((12,180,360)) 
diaz_index = np.zeros((12,180,360)) 
for j in range(12):
    for k in range(180):
        for l in range(360):
            diaz_most_lim_surf[j,k,l] = np.nanmin(diaz_surf[:,j, k, l])
            diaz_index[j,k,l] = _nanargmin(diaz_surf[:,j, k, l], axis=0)
            

In [35]:
import scipy.io as sio
sio.savemat('most_limiting_surf.mat',{'diaz_most_lim_surf':diaz_most_lim_surf, 'pp_most_lim_surf':pp_most_lim_surf,
                                      'diat1_most_lim_surf':diat1_most_lim_surf,'diat2_most_lim_surf':diat2_most_lim_surf,
                                     'diat3_most_lim_surf':diat3_most_lim_surf,'mp1_most_lim_surf':mp1_most_lim_surf,
                                     'mp2_most_lim_surf':mp2_most_lim_surf,'mp3_most_lim_surf':mp3_most_lim_surf,
                                     'mp4_most_lim_surf':mp4_most_lim_surf})
sio.savemat('most_limiting_surf_index.mat', {'diaz_index':diaz_index, 'pp_index':pp_index, 'mp4_index':mp4_index,
                                            'mp3_index':mp3_index, 'mp2_index':mp2_index, 'mp1_index':mp1_index,
                                            'diat1_index':diat1_index, 'diat2_index':diat2_index, 'diat3_index':diat3_index})